In [1]:
!pip install transformers
!pip install datasets transformers[sentencepiece] sacrebleu
!pip install nltk
!pip install torch
!pip install bert_score
!pip install torchtext
!pip install language-tool-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.5 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does 

In [2]:
import pandas as pd
import numpy as np
from datasets import load_dataset
import tensorflow as tf
import torch

In [3]:
# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


In [4]:
# Define dataset path
dataset_path = "cfilt/iitb-english-hindi"
train_dataset = load_dataset(dataset_path, split="train[:1000]")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/1659083 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/520 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2507 [00:00<?, ? examples/s]

In [5]:
english_sentences = [item['en'] for item in train_dataset['translation']]
hindi_sentences = [item['hi'] for item in train_dataset['translation']]

In [6]:
from transformers import pipeline

pipe = pipeline("translation", model="facebook/nllb-200-distilled-600M")

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-distilled-600M").to(device)
translator = pipeline('translation', model=model, tokenizer=tokenizer, src_lang="eng_Latn", tgt_lang='hin_Deva', max_length = 400, device=0 if torch.cuda.is_available() else -1)

config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.55k [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [7]:
translatedEngHin = []
for (index, (eng, hin)) in enumerate(zip(english_sentences, hindi_sentences)):
    print(index+1)
    print("English:", eng)
    prediction = translator(eng)
    translation = prediction[0]['translation_text']
    print("Translated: ", translation)
    print("Original: ", hin)
    print("------------------------------")
    translatedEngHin.append(translation)

comparison = pd.DataFrame({'Original': english_sentences, 'Translated': translatedEngHin})
comparison.to_csv('/content/IITB_Translated_EngHin.csv', index=False)

1
English: Give your application an accessibility workout
Translated:  अपने आवेदन को सुलभता प्रशिक्षण दें
Original:  अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें
------------------------------
2
English: Accerciser Accessibility Explorer
Translated:  एक्सरसाइज़र एक्सेसिबिलिटी एक्सप्लोरर
Original:  एक्सेर्साइसर पहुंचनीयता अन्वेषक
------------------------------
3
English: The default plugin layout for the bottom panel
Translated:  नीचे पैनल के लिए डिफ़ॉल्ट प्लगइन लेआउट
Original:  निचले पटल के लिए डिफोल्ट प्लग-इन खाका
------------------------------
4
English: The default plugin layout for the top panel
Translated:  शीर्ष पैनल के लिए डिफ़ॉल्ट प्लगइन लेआउट
Original:  ऊपरी पटल के लिए डिफोल्ट प्लग-इन खाका
------------------------------
5
English: A list of plugins that are disabled by default
Translated:  प्लगइन्स की सूची जो डिफ़ॉल्ट रूप से अक्षम हैं
Original:  उन प्लग-इनों की सूची जिन्हें डिफोल्ट रूप से निष्क्रिय किया गया है
------------------------------
6
English: Highlight duration
Tra

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Translated:  प्रकाश सीमा का रंग और अस्पष्टता।
Original:  हाइलाइट किए गए सीमांत का रंग और अपारदर्शिता। 
------------------------------
10
English: Highlight fill color
Translated:  भरने का रंग हाइलाइट करें
Original:  भराई के रंग को हाइलाइट करें
------------------------------
11
English: The color and opacity of the highlight fill.
Translated:  हाइलाइट भरने का रंग और अस्पष्टता।
Original:  हाइलाइट किया गया भराई का रंग और पारदर्शिता। 
------------------------------
12
English: API Browser
Translated:  एपीआई ब्राउज़र
Original:  एपीआई विचरक
------------------------------
13
English: Browse the various methods of the current accessible
Translated:  वर्तमान उपलब्ध की विभिन्न विधियों ब्राउज़ करें
Original:  इस समय जिसे प्राप्त किया गया हो, उसकी विभिन्न विधियों (मेथड) में विचरण करें
------------------------------
14
English: Hide private attributes
Translated:  निजी गुणों को छिपाएँ
Original:  निजी गुणों को छिपाएं
------------------------------
15
English: Method
Translated:  विधि
Original:  विधि

In [19]:
# BLEU Score
import nltk
from nltk.translate.bleu_score import corpus_bleu
references = [[hindi] for hindi in hindi_sentences]  # Create reference list of lists
candidates = translatedEngHin  # Model translations
bleu_score = corpus_bleu(references, candidates)
print(f"BLEU score: {bleu_score:.4f}")

BLEU score: 0.4279


In [20]:
# STS Score

!pip install sentence-transformers

# STS Score using SentenceTransformer
from sentence_transformers import SentenceTransformer, util
import torch

# Load the pre-trained model
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
model = model.to('cuda' if torch.cuda.is_available() else 'cpu')

# Encode the original Hindi sentences and the translated sentences
embeddings1 = model.encode(hindi_sentences, convert_to_tensor=True, device=model.device)
embeddings2 = model.encode(translatedEngHin, convert_to_tensor=True, device=model.device)

# Compute cosine similarities between the embeddings
cosine_scores = util.cos_sim(embeddings1, embeddings2)

# Calculate the average STS score
sts_score = cosine_scores.diag().mean().item()
print(f"STS score: {sts_score:.4f}")


STS score: 0.7828


In [21]:
# CHRF

!pip install sacrebleu
from sacrebleu.metrics import CHRF

# Initialize CHRF metric
chrf = CHRF(word_order=1)

# Calculate CHRF score
chrf_score = chrf.corpus_score(translatedEngHin, [hindi_sentences]).score
print(f"CHRF score: {chrf_score:.4f}")


CHRF score: 36.2980


In [ ]:
# METEOR Score
import nltk
from nltk.translate.meteor_score import meteor_score

# Download required NLTK data (if not already downloaded)
nltk.download('wordnet')

# Tokenize the sentences if they're not already tokenized
hindi_sentences_tokenized = [nltk.word_tokenize(sent) for sent in hindi_sentences]
translatedEngHin_tokenized = [nltk.word_tokenize(sent) for sent in translatedEngHin]

# Calculate METEOR score
meteor_scores = [meteor_score([ref], hyp) for ref, hyp in zip(hindi_sentences_tokenized, translatedEngHin_tokenized)]
average_meteor_score = sum(meteor_scores) / len(meteor_scores)

print(f"METEOR score: {average_meteor_score:.4f}")

[nltk_data] Downloading package wordnet to /root/nltk_data...


METEOR score: 0.2773


In [30]:
import nltk
from nltk.lm import KneserNeyInterpolated
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.util import ngrams
import numpy as np

# Download necessary NLTK data
nltk.download('punkt')

def train_model(sentences, n=2):
    tokenized = [['<s>'] + list(map(str.lower, nltk.word_tokenize(sent))) + ['</s>'] for sent in sentences]
    train_data, padded_vocab = padded_everygram_pipeline(n, tokenized)
    model = KneserNeyInterpolated(n)
    model.fit(train_data, padded_vocab)
    return model

def calculate_perplexity(model, sentences, n=2):
    perplexities = []
    for sent in sentences:
        tokens = ['<s>'] + list(map(str.lower, nltk.word_tokenize(sent))) + ['</s>']
        test_data = list(ngrams(tokens, n))
        try:
            pp = model.perplexity(test_data)
            perplexities.append(pp)
        except ValueError:
            pass  # Skip sentences that cause errors
    return np.mean(perplexities) if perplexities else float('inf')

# Train models
hindi_model = train_model(hindi_sentences)
english_model = train_model(translatedEngHin)

# Calculate perplexities
hindi_perplexity = calculate_perplexity(hindi_model, hindi_sentences)
english_perplexity = calculate_perplexity(english_model, translatedEngHin)

print(f"Hindi Perplexity: {hindi_perplexity:.4f}")
print(f"Translated English Perplexity: {english_perplexity:.4f}")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Hindi Perplexity: 11.6234
Translated English Perplexity: 11.7145
